In [1]:
import pandas as pd
import glob
import re
import os

In [2]:
# Folder containing JPM Excel files
DATA_DIR = "./raw_data/"

# Grab all JPM_YYYYQQ.xlsx files
files = sorted(glob.glob(os.path.join(DATA_DIR, "JPM_*.xls")))

print(f"Found {len(files)} files")

Found 35 files


In [3]:
def extract_year_quarter(filename):
    match = re.search(r"(\d{4})Q([1-4])", filename)
    if not match:
        raise ValueError(f"Cannot parse year/quarter from {filename}")

    year = int(match.group(1))
    quarter = int(match.group(2))

    return year, quarter

In [34]:
all_data = []

In [35]:
# def get_sheet_name(year, quarter):
#     # 2010Q3 is the cutoff
#     # if year in (2011, 2012, 2013):
#     #     return "consolidated financial hig"
#     if (year < 2010) or (year == 2010 and quarter < 3):
#         return "jpmorgan chase  co consoli-1"
#     else:
#         return "consolidated financial hig"

def get_skip_rows(year, quarter):
    # if year in (2011, 2012, 2013):
    #     return 6
    if (year == 2020 and quarter < 3):
        return 6
    else:
        return 5

In [36]:
metrics_to_keep = [
    "Total net revenue",
    "Total noninterest expense",
    "Pre-provision profit",
    "Provision for credit losses",
    "Income before income tax expense",
    "Income tax expense",
    "Net income",
    "Common equity Tier 1 (“CET1”) capital ratio(d)",
    "Tier 1 common capital ratio(d)(e)",
    "Tier 1 capital ratio(d)",
    "Total capital ratio(d)",
    "Tier 1 leverage ratio",
    "Loans",
    "Total assets",
    "Deposits",
    "Long-term debt",
    "Common stockholders’ equity",
    "Allowance for credit losses",
    "Allowance for loan losses to total retained loans",
    "Net charge-off rate",
    "Consumer net charge-off rate",
    "Net charge-offs",
    "Nonperforming assets"
]

In [37]:
# df = pd.read_excel(
#             "./raw_data/JPM_2010Q1.xls",
#             sheet_name="consolidated financial hig",
#             skiprows=6  # adjust if needed after checking
#         )
# df.rename(columns={df.columns[0]: "Metric"}, inplace=True)
# # Add time identifiers
# df["Year"] = 2010
# df["Quarter"] = 1
# df["Source"] = "JPM 10-Q"

# df_filtered = df[df["Metric"].isin(metrics_to_keep)]
# # df_long = df_filtered.melt(id_vars="Metric",var_name="Quarter",value_name="Value")
# # df_filtered = df_filtered[df_filtered['Quarter'].str[:7] != "Unnamed"]
# df_filtered = df_filtered[df_filtered.columns.drop(list(df.filter(regex='Unnamed:')))]
# df_filtered

In [38]:
for file in files:
    year, quarter = extract_year_quarter(file)

    try:
        df = pd.read_excel(
            file,
            sheet_name="consolidated financial hig",
            skiprows=get_skip_rows(year, quarter)  # adjust if needed after checking
        )

        # # Keep only relevant columns
        # df = df[COLUMNS_TO_KEEP].copy()

        df.rename(columns={df.columns[0]: "Metric"}, inplace=True)

        # Add time identifiers
        # df["Year"] = year
        # df["Quarter"] = quarter
        # df["Source"] = "JPM 10-Q"

        df_filtered = df[df["Metric"].isin(metrics_to_keep)]
        df_long = df_filtered.melt(id_vars="Metric",var_name="Quarter",value_name="Value")
        df_long = df_long[df_long['Quarter'].str[:7] != "Unnamed"]
        # df_filtered = df_filtered[df_filtered.columns.drop(list(df.filter(regex='Unnamed:')))]

        all_data.append(df_long)
        # merged_df = pd.merge(df_filtered_1, df_filtered, on='Metric', how='left')

    except Exception as e:
        print(f"Error processing {file}: {e}")

In [39]:
jpm_df = pd.concat(all_data, ignore_index=True)
jpm_df = jpm_df[jpm_df['Quarter'].str[1] == "Q"]
jpm_df
# merged_df

,Metric,Quarter,Value
249,Total net revenue,1Q18,27907
250,Total noninterest expense,1Q18,16080
251,Pre-provision profit,1Q18,11827
252,Provision for credit losses,1Q18,1165
253,Income before income tax expense,1Q18,10662
...,...,...,...
969,Common stockholders’ equity,2Q21,253548
970,Allowance for loan losses to total retained loans,2Q21,2.02%
971,Nonperforming assets,2Q21,9802
972,Net charge-offs,2Q21,734


In [10]:
# jpm_df["Quarter_Num"] = jpm_df["Quarter"].str.replace("Q", "").astype(int)

# # Create a proper date column
# jpm_df["Date"] = pd.PeriodIndex(
#     year=jpm_df["Year"],
#     quarter=jpm_df["Quarter_Num"],
#     freq="Q"
# ).to_timestamp()

In [41]:
OUTPUT_PATH = "./output/jpm_retail_portfolio_raw.csv"

jpm_df.to_csv(OUTPUT_PATH, index=False)

print(f"Saved cleaned JPM data to {OUTPUT_PATH}")

Saved cleaned JPM data to ./output/jpm_retail_portfolio_raw.csv
